In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
driver = webdriver.Chrome()
# Fazer login
driver.get('https://sellit-app-web-prod.azurewebsites.net/')

In [3]:
# Feito o login
driver.get('https://sellit-app-web-prod.azurewebsites.net/leads')

In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# --- PRÉ-REQUISITO: A variável 'driver' já deve existir no seu Notebook ---

# Lista para armazenar os resultados para o relatório final
resultados_leads = []

# >>>>>>>>>>>> SELEÇÃO DE LEADS <<<<<<<<<<<<<<<
try:
    limite_input = input("Quantos leads você quer processar? (Deixe em branco e pressione Enter para processar todos): ")
    if limite_input == '':
        LIMITE_DE_LEADS = 9999
        print("Ok, processando todos os leads encontrados.")
    else:
        LIMITE_DE_LEADS = int(limite_input)
except ValueError:
    print("Entrada inválida. Usando o limite padrão de 2000.")
    LIMITE_DE_LEADS = 2000
# ---------------------------------------------

wait = WebDriverWait(driver, 120)
print(f"--- Iniciando a automação (limite de {LIMITE_DE_LEADS} leads) ---")

leads_processados_total = 0

# --- LOOP PRINCIPAL COM PAGINAÇÃO ---
while leads_processados_total < LIMITE_DE_LEADS:
    try:
        print(f"\n--- Verificando página: {driver.current_url} ---")
        # Espera que a tabela esteja presente e visível
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'table thead ~ thead')))
        linhas = driver.find_elements(By.CSS_SELECTOR, 'table thead ~ thead')
        print(f"Sucesso! {len(linhas)} leads encontrados nesta página.")

        for i in range(len(linhas)):
            if leads_processados_total >= LIMITE_DE_LEADS:
                print(f"\nLimite de {LIMITE_DE_LEADS} leads atingido. Interrompendo o processo.")
                break

            linha_atual = driver.find_elements(By.CSS_SELECTOR, 'table thead ~ thead')[i]
            nome_lead = f"Lead na posição {i+1}"

            try:
                # --- CORREÇÃO DE FOCO E TIMING ---
                # 1. Rola a linha para o centro da tela
                driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", linha_atual)
                # 2. Rola um pouco mais para baixo (150 pixels) para garantir que o cabeçalho não atrapalhe
                driver.execute_script("window.scrollBy(0, -150);")
                # 3. Pausa explícita de 1 segundo para a interface gráfica assentar.

                
                expira = linha_atual.find_element(By.CSS_SELECTOR, 'td:nth-child(13)').text

                print(f"\nAnalisando Lead Status: {expira}")
                expirado = expira.split()
                expirado_int = int(expirado[0])

                if expirado_int > 30:
                    print(f'O lead tem mais de 30 dias para expirar (pulado)')
                    
                    continue

                print(f"--> ATUALIZANDO lead ''...")

                # Usando o clique normal que você confia, agora que a visão está limpa
                botao_acao = wait.until(EC.element_to_be_clickable(linha_atual.find_element(By.CSS_SELECTOR, 'a[aria-label="Criar tarefa"]')))
                botao_acao.click()

                # O restante do seu código que já funciona...
                wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.MuiDialog-root.sc-imWYAI.dMYvXE')))
                
                seletor_seta_tarefa = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(1) > div > div > div > div > div > button.MuiButtonBase-root.MuiIconButton-root.MuiAutocomplete-popupIndicator'
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_seta_tarefa))).click()
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li#combo-box-demo-option-2'))).click()
                
                texto_descricao = f'Ofertei, Home Ipiranga, United, Addres, Acuruí, Home boutique, Palazzo, Duet, Edition, High line, Residences, Sence e lançamento na Barão do Triunfo e outro na vila mascote. Quer ser avisado de todos os lançamentos! Totalmente aberto as localizações.'
                seletor_descricao = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(5) > div > div > div > textarea:nth-child(1)'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_descricao))).send_keys(texto_descricao)
                seletor_checkbox_xpath = '/html/body/div[5]/div[3]/div/div[2]/div/h2/div[1]/div[6]/div[2]/label/span[1]/span[1]/input'
                checkbox = wait.until(EC.presence_of_element_located((By.XPATH, seletor_checkbox_xpath)))
                checkbox.click()
                seletor_seta_resultado = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(7) > div > div > div > div > div > button.MuiButtonBase-root.MuiIconButton-root.MuiAutocomplete-popupIndicator'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_seta_resultado))).click()
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li#combo-box-demo-option-0'))).click()
                texto_resultado = 'Lead contatado, em acompanhamento.'
                seletor_texto_resultado = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-ihgnxF.darqTH > div:nth-child(8) > div > div > div > textarea:nth-child(1)'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_texto_resultado))).send_keys(texto_resultado)
                seletor_salvar = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-fHjqPf.jyGuqJ > button.MuiButtonBase-root.MuiButton-root.MuiButton-text.sc-feUZmu.dwrURg'
                
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_salvar))).click()
                seletor_fechar = 'body > div.MuiDialog-root.sc-imWYAI.dMYvXE.shake-false > div.MuiDialog-container.MuiDialog-scrollPaper > div > div.MuiDialogContent-root.sc-aXZVg.eCBXCQ > div > h2 > div.sc-fHjqPf.jyGuqJ > button.MuiButtonBase-root.MuiButton-root.MuiButton-outlined.sc-dAbbOL.dIjAJu'
                time.sleep(1)
                wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, seletor_fechar))).click()
                

                
                leads_processados_total += 1

            except Exception as e:
                print(f"!!!!!!!! ERRO ao processar o lead '': {e}")
                
                break

        if leads_processados_total >= LIMITE_DE_LEADS:
            break

        # --- LÓGICA DE PAGINAÇÃO ---
        try:
            driver.refresh()
            time.sleep(4)
        except NoSuchElementException:
            print("\nNão foi possível encontrar o botão 'próximo'. Fim de todos os leads.")
            break

    except TimeoutException:
        print('Passou mais de 2 minutos não achou')
        print("\nERRO CRÍTICO: Tabela de leads não foi encontrada na página. O loop de paginação vai parar.")
        break

# --- FIM DE TODOS OS LOOPS ---

# --- 3. CRIAR O RELATÓRIO CSV ---



print("\n\nFim da execução. O navegador continua aberto.")

Ok, processando todos os leads encontrados.
--- Iniciando a automação (limite de 9999 leads) ---

--- Verificando página: https://sellit-app-web-prod.azurewebsites.net/leads#/ ---
Sucesso! 100 leads encontrados nesta página.

Analisando Lead Status: 30 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead ''...

Analisando Lead Status: 28 dias
--> ATUALIZANDO lead '